In [9]:
//  Copyright (c) 2020 Patrick Diehl
//
//  SPDX-License-Identifier: BSL-1.0
//  Distributed under the Boost Software License, Version 1.0. (See accompanying
//  file LICENSE_1_0.txt or copy at http://www.boost.org/LICENSE_1_0.txt)

In [2]:
#include <iostream>
#include<run_hpx.cpp>
#include<hpx/include/lcos.hpp>
#include<hpx/include/parallel_for_loop.hpp>
#include<atomic>

# Numerical integration

The trapezoidal rule can be used to approximate the definite integral
\begin{align*}
\int\limits_a^b f(x) dx \approx \frac{h}{2} \sum\limits_{k=1}^N (f(x_{k-1}) + f(x_k))
\end{align*}
assuming a uniform grid in the interval $[a,b]$ with the grid size $h=\frac{b-a}{N}$.

### Define the function $f(x)=x^2$ to integrate

In [3]:
double f(double x){
    return x*x;
}

### Define the integration interval

In [4]:
size_t N = 100;
double a = 0;
double b = 2;

double h = (b-a) / N;

# Serial implementation of the integration

In [5]:
double area = 0; 

for(size_t i = 1; i <= N; i++){
    
    area += f(h*(i-1))+f(h*i);
    
}

area *= h/2;

std::cout << area << std::endl;

2.6668


(std::basic_ostream<char, std::char_traits<char> >::__ostream_type &) @0x7f97275cf500


## Parallel implementation I (parallel algorithms)

In [6]:
run_hpx([](){

double area = 0;
hpx::lcos::local::mutex m;

hpx::for_loop(
	hpx::execution::par, 
	0, 
	N,
	[&](boost::uint64_t i)
		{
            m.lock();
		    area += f(h*(i-1))+f(h*i);
            m.unlock();
		}
	);

area = area * h /2 ;

std::cout << area << std::endl;

});

1.36113e+33


(void) @0x7f971d75fe48


## Parallel implementation II (futurization)

In [7]:
double integrate(size_t begin, size_t end,double h){
 
    double area = 0;
    
    
    for(size_t i = begin; i <= end; i++){
         area += f(h*(i-1))+f(h*i);
    } 
    
    return area;
}

In [8]:
run_hpx([](){

auto f1 = hpx::async(integrate,1,49,h);
auto f2 = hpx::async(integrate,50,100,h);

std::cout << h/2*(f1.get() + f2.get()) << std::endl;

});

2.6668


(void) @0x7f971d75fe48
